In [1]:
import torch as tc
import pandas as pd
import numpy as np
from src.bbox import BBox
import open3d as otd
from tqdm import tqdm
import matplotlib.pyplot as plt
otd_vector3d = otd.utility.Vector3dVector
from src.star.star import STAR
from src.curve_utils import CurveUtils
from src.curve_generator import CurveGenerator
from src.mesh_manipulation import save_obj
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
genders = ['female', 'male']
male = ['male']
female = ['female']

In [2]:
their_semantics = [
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    #'neck_waist': 5
}

In [3]:
subdivided_bodies = tc.load('data/subdivided_bodies.pt')
measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures.index = measures['Subject']
measures = measures[measures['Measuring station'] == "MOVE4D"]

mfd_gender_measures = dict()
for gender in genders:
    mfd_gender_measures[gender] = measures[measures['Sex'] == gender]
    mfd_gender_measures[gender] = mfd_gender_measures[gender][their_semantics]
    mfd_gender_measures[gender].columns = our_semantic
    mfd_gender_measures[gender]['stature'] *= 1000

In [4]:
selected_subjects = dict()
selected_measures = dict()

for gender in genders:
    gender_measures = measures[measures['Sex'] == gender]
    temp_measures = gender_measures[their_semantics].iloc[::2]
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [5]:
body = subdivided_bodies['vertices']['female'][selected_subjects['female']]
template = subdivided_bodies['faces']['female']+1
# save_obj(path="output/female_body.obj", pontos=body, faces=template)

In [6]:
gender_curves = dict()
for gender in genders:
    print(f'SEGMENTING {gender.upper()} BODIES', end='')
    faces = subdivided_bodies['faces'][gender].to(device)
    bodies = subdivided_bodies['vertices'][gender]
    body = bodies[selected_subjects[gender]].to(device)
    measures = selected_measures[gender]
    result = CurveGenerator.get_curves(body, measures, faces, device, gender)
    gender_curves[gender] = result[0]
tc.save(gender_curves, "data/gender_curves.zip")

SEGMENTING FEMALE BODIES

processing body:  98%|█████████▊| 44/45 [00:43<00:00,  1.00it/s]


SEGMENTING MALE BODIES

processing body:  98%|█████████▊| 44/45 [00:47<00:01,  1.07s/it]


In [7]:
gender_measures = dict()
for gender in genders:
    print(f'MEASURING {gender.upper()} BODIES', end='')
    gender_measures[gender] = [[]]*5
    for segments_index, curves_segments in enumerate(gender_curves[gender]):
        gender_measures[gender][segments_index] = []
        for curves_index, curves in enumerate(tqdm(curves_segments)): # bust, torso, leg, arm, neck
            
            gender_measures[gender][segments_index].append([])
            for body in subdivided_bodies['vertices'][gender]:
                position = CurveUtils.generate_positions(tc.FloatTensor(curves).to(device), body.to(device))
                calculated_measures = CurveUtils.calculate_distances(position)
                gender_measures[gender][segments_index][curves_index].append(calculated_measures)

            calculated_measures = gender_measures[gender][segments_index][curves_index]
            gender_measures[gender][segments_index][curves_index] = tc.FloatTensor(calculated_measures)
        gender_measures[gender][segments_index] = tc.row_stack(gender_measures[gender][segments_index])

tc.save(gender_measures, "data/calculated_measures.zip")


MEASURING FEMALE BODIES

100%|██████████| 640/640 [00:46<00:00, 13.64it/s]


MEASURING MALE BODIES

100%|██████████| 680/680 [00:48<00:00, 13.90it/s]


In [8]:
gender_calculated_measures = tc.load("data/calculated_measures.zip")

In [9]:
gender_measures_wc = dict()
for gender in genders:
    gender_measures_wc[gender] = []
    for index in range(0,5):
        gender_measures_wc[gender].append(gender_measures[gender][index])
        if index == 1:
            gender_measures_wc[gender].append(gender_measures[gender][index])
gender_measures = gender_measures_wc

In [13]:
best_gender_measures = dict() ## caso médio
for idx, gender in enumerate(genders):
    best_gender_measures[gender] = []
    for index, curve in enumerate(our_semantic[:-2]):
        measured = gender_measures[gender][index].T[::2].T
        ground_truth = mfd_gender_measures[gender][curve][::2]
        result = (measured - tc.FloatTensor(ground_truth/10).unsqueeze(0)).abs()
        min_rows_values = result.mean(1)
        min_columns_values, min_columns_indices = min_rows_values.min(0)
        best = min_columns_indices
        best_gender_measures[gender].append((
            best.numpy(),
            result[best].min().numpy(),
            result[best].max().numpy(),
            result[best].mean().numpy(),
            result[best].std().numpy()
        ))
gender_results = {
    'male': pd.DataFrame(best_gender_measures['male'], columns=['best','min', 'max', 'mean', 'std'], index=our_semantic[:-2]),
    'female': pd.DataFrame(best_gender_measures['female'], columns=['best','min', 'max', 'mean', 'std'], index=our_semantic[:-2])
}
print("male errors:")
print(gender_results['male'])
print("\nfemale errors:")
print(gender_results['female'])

tc.save(gender_results, "data/best_gender_curves.zip")

male errors:
                 best           min        max        mean        std
bust_chest_girth   19   0.048316956   4.917015   1.3688946  1.0689367
waist_girth        33   0.008003235  2.1719513   0.6310151  0.5437107
hip_girth          13  0.0023117065  1.2593384  0.45648277  0.3605611
thigh_girth        54   0.004257202  1.8026123  0.70295495  0.5112756
upper_arm_girth    38   0.002937317  0.7723694    0.318335  0.2103261
neck_girth        496  0.0025863647  2.4173927   0.7392074  0.5782629

female errors:
                 best          min        max        mean         std
bust_chest_girth   14   0.16119385   4.739174   1.8618696   1.1967944
waist_girth        30  0.014572144  3.6633835   1.0967818   0.9472004
hip_girth          10  0.057029724  5.4276123   1.2659556   1.1800628
thigh_girth        47  0.019634247  3.2767715  0.86906433  0.68342984
upper_arm_girth    34  0.030994415  1.6408348   0.6151817  0.38860407
neck_girth        418  0.016311646  1.5304298   0.5129456  0.